In [2]:
#!pip install "gradio<4.0.0"
#!pip install "transformers[torch]>=4.23" gradio faster-whisper sentencepiece
#!pip install openai-whisper
#!pip install torch transformers
#!pip install openai-whisper
#!pip install torch transformers requests gradio


# Transcripción de Audio con Whisper y Clasificación Zero-Shot

Este proyecto demuestra cómo construir aplicaciones web con modelos Transformer para procesamiento del lenguaje natural, específicamente para transcripción de audio y clasificación de texto zero-shot.



## Contenido

- Ejemplos de cómo construir aplicaciones web con modelos Transformer para Procesamiento del Lenguaje Natural
  - Ejemplo: Construir una interfaz de usuario para transcripción
  - Ejemplo II: Clasificación de texto Zero-shot
  - Ejercicios prácticos
- Demo: Clasificador de audios zero-shot    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


## Ejemplos de cómo construir aplicaciones web con modelos Transformer para Procesamiento del Lenguaje Natural

Para que las demostraciones se ejecuten rápidamente, debes usar una GPU.

Actívala yendo a "Entorno de ejecución / Cambiar tipo de entorno de ejecución". Luego selecciona la GPU.

**HAZ ESTO ANTES DE EJECUTAR CUALQUIER CELDA**

Después de activar la GPU, vamos a instalar las bibliotecas requeridas:


In [3]:
#!pip install --q "transformers[torch]>=4.23" gradio faster-whisper sentencepiece

## <a id='toc2_1_'></a>[Ejemplo I: Construir una interfaz de usuario para transcripción](#toc0_)

Vamos a construir una demo de [Gradio](https://gradio.app) para el reconocimiento automático de voz.

Usaremos [faster-whisper](https://github.com/guillaumekln/faster-whisper) para una transcripción multilingüe eficiente.

Comencemos cargando el modelo:


In [4]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device=}")

Using device='cpu'


In [5]:
from faster_whisper import WhisperModel
model_size = "large-v3-turbo"
# model_size = "tiny"
model = WhisperModel(model_size, device=device, compute_type="float32")

def transcribe_speech(filepath:str) -> str:
    """
    Transcribe the audio given by a filepath (which can be an URL)

    Parameters
    ==========
    filepath: str
        Path to audio file to transcribe

    Returns
    =======
    text: str
        Transcribed text
    """
    segments, info = model.transcribe(filepath,
                                      task ="transcribe",
                                      vad_filter=True,
                                      without_timestamps=True,
                                      beam_size=5)
    text = " ".join(seg.text for seg in segments)

    return text

Lo aplicamos a un ejemplo:

In [6]:
transcribe_speech("https://www.voiceoverbird.co.uk/wp-content/uploads/2021/06/Amazon_Eastwood_EMA3_Radio_30_Aug20.mp3")

" At Amazon, it's our warehouse associates that support us to ensure we deliver the full package. We've got plenty of opportunities at our new site in Eastwood, so come and join our growing team and be part of something bigger. You'll receive £9.50 per hour, plus overtime and night shift allowances. Become an Amazon Associate today, with no experience required. Apply now at amazonjobs.co.uk  Thank you."

Usaremos Gradio para lanzar dos opciones para subir audio en nuestra demo:
* una que permite el uso de micrófono
* la otra subida de archivos


In [7]:
import gradio as gr

examples = ["https://www.voiceoverbird.co.uk/wp-content/uploads/2021/06/Amazon_Eastwood_EMA3_Radio_30_Aug20.mp3",
            "https://www.voiceoverbird.co.uk/wp-content/uploads/2020/04/supermarketinstructions.mp3",
            "https://www.voiceoverbird.co.uk/wp-content/uploads/2020/04/straighttv-narration.mp3",
            "https://upload.wikimedia.org/wikipedia/commons/d/d6/Audio_Derrumbe_de_bloque_oce%C3%A1nico_en_Australia_generar%C3%ADa_un_gran_tsunami_-_Wikinoticias.ogg"]

mic_transcribe = gr.Interface(
    title = "Tanscripción de audio",
    fn=transcribe_speech,
    inputs=gr.Audio(sources=["microphone","upload"], type="filepath", label="Audio de entrada"),
    outputs=gr.Textbox(label="Transcripción"),
    examples = examples
)

mic_transcribe.launch(debug=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## <a id='toc2_2_'></a>[Ejemplo II: Clasificación de texto Zero-shot](#toc0_)

Importamos las librerías para cargar el modelo:

In [8]:
from transformers import pipeline
import torch
import gradio as gr


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device=}")

classifier = pipeline("zero-shot-classification",
                      # model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
                      model="facebook/bart-large-mnli",
                      device=device)

Using device='cpu'


Device set to use cpu


Comprobamos que funciona bien algunos ejemplos:
* Ejemplo en el que el texto y las etiquetas están en inglés
* Ejemplo en el que el texto y las etiquetas están en español
* Ejemplo mezclando inglés y español

Sorprendentemente, estos modelos son casi agnósticos al idioma utilizado!

In [9]:
sequence_to_classify = "I'm really exited about the movie I'll watch next month: Dune II."
candidate_labels = ["politics", "finance", "entertainment", "environment"]

output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
output

{'sequence': "I'm really exited about the movie I'll watch next month: Dune II.",
 'labels': ['entertainment', 'environment', 'finance', 'politics'],
 'scores': [0.8937325477600098,
  0.07902202010154724,
  0.015463797375559807,
  0.011781622655689716]}

In [10]:
sequence_to_classify = "La película empezó muy bien, al final fue una decepción."
candidate_labels = ["positivo", "negativo","neutro"]

output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
output

{'sequence': 'La película empezó muy bien, al final fue una decepción.',
 'labels': ['negativo', 'neutro', 'positivo'],
 'scores': [0.9461797475814819, 0.042328108102083206, 0.011492135003209114]}

In [11]:
sequence_to_classify = "I'm really exited about the movie I'll watch next month: Dune II."
candidate_labels = ["politica", "economia", "entretenimiento", "medio ambiente"]

output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
output

{'sequence': "I'm really exited about the movie I'll watch next month: Dune II.",
 'labels': ['entretenimiento', 'medio ambiente', 'economia', 'politica'],
 'scores': [0.6981919407844543,
  0.12665197253227234,
  0.11840981990098953,
  0.0567462258040905]}

In [12]:
sequence_to_classify = "Conduce por la montaña con comodidad y estilo en el CLA Coupé. Su diseño deportivo y su ancho de vía no solo llaman la atención, sino que mejoran el dinamismo y la estabilidad."
candidate_labels = ['Mercedes', 'BMW', 'Ford','Audi']

output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
output

{'sequence': 'Conduce por la montaña con comodidad y estilo en el CLA Coupé. Su diseño deportivo y su ancho de vía no solo llaman la atención, sino que mejoran el dinamismo y la estabilidad.',
 'labels': ['Mercedes', 'Ford', 'Audi', 'BMW'],
 'scores': [0.7695899605751038,
  0.10042674839496613,
  0.089850053191185,
  0.04013325646519661]}

In [13]:
sequence_to_classify = "La película estuvo normal, no me encantó, pero tampoco me aburrió."
candidate_labels = "positivo negativo neutro".split()
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
output

{'sequence': 'La película estuvo normal, no me encantó, pero tampoco me aburrió.',
 'labels': ['neutro', 'negativo', 'positivo'],
 'scores': [0.6839929819107056, 0.30100110173225403, 0.015005908906459808]}

Definimos una función para hacer más sencillo el uso en la demo

In [14]:
def zero_shot_classification(text_input:str, candidate_labels:str) -> dict[str,float]:
    """
    Performs a zero-shot text classification, for a given set of candidate_labels.
    Parameters
    ==========
    text_input: str
    candidate_labels: str
        Labels separated by comma, it makes it easier to use in the gradio demo

    Returns
    =======
    outputs: dict[str,float]
        Dictionary with the predicted probabilities of each label
    """
    labels = [label.strip(' ') for label in candidate_labels.split(',')]
    prediction = classifier(text_input, labels)
    print(prediction)

    # Generamos un diccionario label -> score
    output = dict(zip(prediction['labels'], prediction['scores']))

    return output

In [15]:
zero_shot_classification(sequence_to_classify, ",".join(candidate_labels))

{'sequence': 'La película estuvo normal, no me encantó, pero tampoco me aburrió.', 'labels': ['neutro', 'negativo', 'positivo'], 'scores': [0.6839929819107056, 0.30100110173225403, 0.015005908906459808]}


{'neutro': 0.6839929819107056,
 'negativo': 0.30100110173225403,
 'positivo': 0.015005908906459808}

In [16]:
zero_shot_classification("Conduce por la montaña con comodidad y estilo en el CLA Coupé. Su diseño deportivo y su ancho de vía no solo llaman la atención, sino que mejoran el dinamismo y la estabilidad.", "BMW, Ford, Audi, Mercedes")

{'sequence': 'Conduce por la montaña con comodidad y estilo en el CLA Coupé. Su diseño deportivo y su ancho de vía no solo llaman la atención, sino que mejoran el dinamismo y la estabilidad.', 'labels': ['Mercedes', 'Ford', 'Audi', 'BMW'], 'scores': [0.769589900970459, 0.10042674094438553, 0.089850053191185, 0.04013325646519661]}


{'Mercedes': 0.769589900970459,
 'Ford': 0.10042674094438553,
 'Audi': 0.089850053191185,
 'BMW': 0.04013325646519661}

Finalmente, lanzamos la demo de Gradio usando un bloque.
Para poder ejecutar otras celdas, debes detener esta.

Hemos agregado dos ejemplos a la demostración.

In [17]:
examples = [["One day I will see the world", "travel, live, die, future"],
            ["The movie started quite well, but at the end it was a disaster.", "positive feeling, negative feeling, neutral feeling"],
            ["La película empezó muy bien, pero al final fue una decepción.", "positivo, negativo, neutro"]]


demo = gr.Interface(fn=zero_shot_classification,
                    inputs=[gr.Textbox(label="Texto a clasificar"),
                            gr.Textbox(label="Etiquetas candidatas")],
                    outputs=gr.Label(label="Clasificación"),
                    title="Clasificación de textos zero-shot",
                    examples=examples)
demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


<span style="background-color: red; color: white; padding: 10px; display: block;">

## <a id='toc2_3_'></a>[Ejercicios:](#toc0_)
* Prueba un par de casos de uso de esta tecnología.
* En alguno de ellos, añade más etiquetas posibles. ¿Cómo cambian los resultados?
* En alguno de ellos, sustituye las etiquetas por sinónimos. ¿Cómo cambian los resultados?

Esto es un ejemplo de *prompt engineering*.
</span>


# <a id='toc3_'></a>[Construye una demo para un clasificador de audios zero-shot](#toc0_)



In [18]:
import gradio as gr
from transformers import pipeline
from faster_whisper import WhisperModel

whisper_model = WhisperModel(
    "small", 
    device="cuda" if torch.cuda.is_available() else "cpu",
    compute_type="float32"  # Change from float16 to float32
)
# Pipeline zero-shot
zero_shot_classifier = pipeline("zero-shot-classification",
                                 model="facebook/bart-large-mnli",
                                 device=0 if torch.cuda.is_available() else -1)

# Etiquetas por caso de uso
etiquetas = {
    "Emociones": ["alegría", "tristeza", "miedo", "enojo", "sorpresa"],
    "Intenciones": ["invitar a alguien", "expresar una opinión", "compartir una experiencia personal", "contar una historia", "hacer una propuesta"]

}

def transcribe_y_clasificar(audio, caso):
    if audio is None:
        return "Por favor, sube o graba un audio."

    # Transcripción con Whisper
    segments, info = whisper_model.transcribe(audio, beam_size=5)
    texto = " ".join([seg.text for seg in segments])
    
    # Clasificación zero-shot
    resultado = zero_shot_classifier(texto, etiquetas[caso])

    # Construir respuesta
    salida = f"### Transcripción automática:\n\n{texto}\n\n"
    salida += f"### Clasificación ({caso}):\n"
    for label, score in zip(resultado["labels"], resultado["scores"]):
        salida += f"- **{label}**: {score:.2f}\n"
    return salida

# Interfaz Gradio
gr.Interface(
    fn=transcribe_y_clasificar,
    inputs=[
        gr.Audio(type="filepath", label="Graba o sube un audio"),
        gr.Radio(choices=["Emociones", "Intenciones"], label="Selecciona el caso de uso")
    ],
    outputs=gr.Markdown(),
    title="Ejercicio 4 – Clasificador Zero-Shot desde Audio",
    description="Este ejercicio permite grabar o subir un audio, transcribirlo automáticamente y clasificar la intención o emoción con un modelo zero-shot.",
    examples=[
    ["https://audio-lingua.ac-versailles.fr/IMG/mp3/conversation_telephonique_maria_lucia.mp3", "Intenciones"],
    ["https://audio-lingua.ac-versailles.fr/IMG/mp3/el_cura_enfadado.mp3", "Emociones"],
    ["https://audio-lingua.ac-versailles.fr/IMG/mp3/tiburones.mp3", "Emociones"],
    ["https://audio-lingua.ac-versailles.fr/IMG/mp3/Sioban_what_scares_me_the_most_driving_2.mp3", "Emociones"],
    ["https://audio-lingua.ac-versailles.fr/IMG/mp3/911.mp3", "Emociones"],
]
).launch()


Device set to use cpu


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Using existing dataset file at: .gradio/flagged/dataset2.csv
